# Тестовое задание по Data Science (9-й набор)

### Задача: Создать рекомендательную систему для генерации рекомендаций об образовательных программах (курсы, книги, статьи).

### Очистка данных


Для очистки данных необходимо использовать инструменты, предлагаемые библиотекой Pandas.

1. Получить информацию о данных (`df.info()`) для понимания к какому типу данных относятся те или иные значения в таблице (числовые данные или данные типа object).
Настоящий шаг необходим для дальнейшего преобразования данных в случае наличия пропусков.
2. Узнать сколько пропусков содержится в данных (`df.isnull().sum()`). Настоящий метод вернет количество пропусков в каждом из признаков датафрейма.
Также можно узнать процентное соотношение пропусков в каждом признаке к непропущенным значениям. 

```python
col_null_precent = df.isnull().mean() * 100
col_with_null = col_null_precent[col_null_precent > 0].sort_values(ascending=False)
```

Настоящий код вернет количество пропусков в процентах в каждом из признаков датафрейма в порядке убывания. В случае, если процент пропусков в каждом конкретном признаке превышает 30%-40%, то можно удалить столбец или столбцы используя следующий код:

```python
df.drop('Наименование признака', axis =1, inplace=True)
df.drop(['Наименования признаков в виде списка'], axis=1, inplace=True)
```

3.  Для всех тех признаков, где количество пропущенных значений в процентах меньше вышеуказанного диапазона, можно удалить строки с пропусками используя метод dropna(). Однако в таком случае можно потерять важные данные, поэтом можно заполнить пропущенные значения константами. Категориальные признаки, содержащие пропуски, можно заполнить модальным значением:

```python
df['Наименование признака'] = df['Наименование признака'].fillna(df['Наименование признака'].mode()[0])
```

Пропуски в числовых признаках можно заполнить средним значением, медианой или модальным значением. Код замены будет аналогичен предыдущему, за исключением агрегирующих методов (mean(), median()).
Также, необходимо обработать выбросы и дубликаты в данных. 

4. Для обработки выбросов можно воспользоваться методом Тьюки.

```python
# пустое множество для хранения индексов строк с выбросами 
outliers = set()
# цикл по каждому признаку для поиска выбросов 
for column in df.columns:
    # вычисление первого и второго квартиля
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    # вычисление межквартильного расстояния
    iqr = q3 - q1
    # определение верхней и нижней границы
    lb = q1 - 1.5 * iqr
    ub = q3 + 1.5 * iqr
    # находим индексы строк с выбросами
    col_outliers = df[(df[column] < lb) | (df[column] > ub)].index
    # обновляем множество для хранения индексов строк с выбросами 
    outliers.update(col_outliers)
# удаляем выбросы    
df.drop(outliers, inplace=True)
```

Если при удалении выбросов в датафрейме сбился порядок индексов можно воспользоваться методом reset_index() с параметром inplace=True. При использовании этой функции создается признак index который нужно будет удалить.

```python
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

```
5. Для поиска и удаления дубликатов можно воспользоваться методом duplicated() с параметром subset = column. Дубликаты будут искаться во всех колонках датафрейма.

```python
dupl = df.duplicated(subset = column)
df[dupl]
```
Чтобы удалить дубликаты можно воспользоваться методом drop_duplicates с параметрами subset = column и inplace=True. Дубликаты будут удалены во всех колонках датафрейма.

```python
df.drop_duplicates(subset = column, inplace=True)
```

### Выделение признаков (features).

Для каждого навыка пользователя и для каждого курса можно создать признаки уровня навыка (junior_grade, midle_grade и senior_grade), для целей дальнейшего сопоставления текущего уровня пользователя и уровня курса, который будет развивать навык:

``` python
user_df['junior_grade_python'] = user_df['python'].apply(lambda x: 1 if x <= 2 else 0)
user_df['midle_grade_python'] = courses['python'].apply(lambda x: 1 if 2 <= x <= 4 else 0)
user_df['senior_grad_python'] = courses['python'].apply(lambda x: 1 if x >= 4 else 0)  
```

Аналогичный код для данных по курсам.

В качестве дополнительных данных для таблицы с пользователями было бы полезно знать цель пользователя, то есть каким уровнем навыка он бы хотел обладать. В таком случае модель могла бы сравнивать текущий уровень и желаемый уровень навыка пользователя с курсами, которые способствуют получению такого уровня. Например, если у пользователя уровень навыка 0, а желаемый уровень 2, то модель предлагала бы пользователю курсы с уровнями от 0 до желаемого уровня. Также, поскольку в таблице с обучающими материалами указана цена, в таблицу с пользователями, в качестве дополнительных данных, можно добавить бюджет, для целей сопоставления стоимости обучающего материала с бюджетом пользователя.

В таблицу с обучающими материалами, в качестве дополнительной информации, можно добавить рейтинг материала, чтобы пользователь мог оценить уровень содержания обучающего материала. Думаю, также важен признак времени освоения навыка.

### Построение модели.

Ознакомившись с материалами, представленными в открытых источниках, могу сказать что, существует три подхода (метода) реализации рекомендательной системы:
1. Контентно-ориентированная фильтрация - когда рекомендации происходят на основе содержания элементов. Например, сопоставление навыков и уровня их владения, предоставляемых обучающими материалами, с навыками и уровнем их владения пользователем;
2. Коллаборативная фильтрация - когда рекомендации происходят на основе схожести пользователей или схожести элементов. Например, пользователю рекомендуются обучающие материалы, которые понравились другим, похожим на пользователя, пользователям или пользователю рекомендуются обучающие материалы, которые схожи с уже изученными им материалами;
3. Гибридные методы - объединение двух предыдущих методов в один.

Мне представляется, что в настоящем случае, предпочтительно использовать именно гибридные методы, поскольку это позволит максимально увеличить функционал рекомендательной системы, поскольку пользователю будут предлагаться материалы и на основе навыков пользователя и на основе схожести между пользователями, а также на основе схожести самих обучающих материалов.

Для реализации данного подхода можно воспользоваться инструментами библиотеки sklearn, такими как косинусное сходство (`from sklearn.metrics.pairwise import cosine_similarity`) для коллаборативной фильтрации, евклидово расстояние (`from sklearn.metrics.pairwise import euclidean_distances`) для контентно-ориентированной фильтрации. После необходимо объединить результаты работы двух этих методов.

Также можно использовать специальную библиотеку для построения гибридной рекомендательной системы - `LightFM`.

В рамках программы моего курса обучения, подробные теоретические знания и практические навыки реализации рекомендательных систем будут освоены мной в ближайшее время (в ноябре текущего года).

### Оценка качества модели

В качестве метрик качества можно использовать Recall для понимания того, сколько действительно релевантных материалов было предложено пользователю и Precision - оценка релевантности рекомендаций.
Также можно использовать F1-score (баланс между Precision и Recall) для получения более точного представления о качестве модели.   

В качестве ошибки первого рода (ложно-положительная ошибка) может быть ситуация, при которой модель рекомендует пользователю обучающий материал не соответствующий его целям, уровню навыков или бюджету и, как пример, предлагает более сложный курс новичку или продвинутому пользователю - материалы начального уровня. Ошибкой первого рода также будет являться рекомендация модели пользователю материалов не соответствующих интересу пользователя.
Ошибками второго рода (ложно-отрицательная ошибка) могут быть случаи, при которых модель не предлагает пользователю обучающий материал, который точно подходит пользователю, исходя из его навыков и интересов. Также модель может не предложить более сложный материал пользователю, предполагая, что последний может не справится с его освоением, хотя на самом деле, пользователь способен это сделать.

Для борьбы с ошибками можно обновлять данные о пользователях и об обучающих материалах, для предоставления более точных рекомендаций. Также, предполагаю, что можно внедрить функционал обратной связи от пользователей, чтобы последние могли указывать на не релевантность рекомендаций.